In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from descartes.patch import PolygonPatch
import contextily as ctx
from shapely.geometry import Point,LinearRing, Polygon ,MultiLineString
import mplleaflet
from geopandas import GeoSeries
import datetime
import time
import numpy as np
import plotly.express as px
from keplergl import KeplerGl as keplergl
import ipywidgets as widget

In [2]:
df = pd.read_csv('tu_mobility.csv', delimiter = ',')
df

,latitude,longitude,datetime,id
0,13.868754,100.409435,2019-07-07 22:57:46,1319
1,13.884972,100.506767,2019-07-08 05:52:01,1316
2,13.884972,100.506767,2019-07-08 05:52:04,1316
3,13.884972,100.506767,2019-07-08 05:52:05,1316
4,13.884972,100.506767,2019-07-08 05:52:06,1316
...,...,...,...,...
4256984,13.966694,100.712861,2019-09-01 14:00:05,1573
4256985,14.299056,100.425677,2019-09-01 16:34:45,117
4256986,13.827038,100.622365,2019-09-01 17:02:56,1373
4256987,13.827038,100.622365,2019-09-01 17:02:57,1373


In [3]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['day'] = df['datetime'].dt.day_name()

In [4]:
df['month'] = df['datetime'].dt.month_name()

In [5]:
def find_period(hour,minute):
    return(
        "morning" if 5<=hour<=11
         else
        "afternoon" if 12<=hour<=16
        else
        "evening" if 17<=hour <=19
        else
        "night"       
    )

In [6]:
df['period'] = np.vectorize(find_period)(df['datetime'].dt.hour,df['datetime'].dt.minute)
df

,latitude,longitude,datetime,id,day,month,period
0,13.868754,100.409435,2019-07-07 22:57:46,1319,Sunday,July,night
1,13.884972,100.506767,2019-07-08 05:52:01,1316,Monday,July,morning
2,13.884972,100.506767,2019-07-08 05:52:04,1316,Monday,July,morning
3,13.884972,100.506767,2019-07-08 05:52:05,1316,Monday,July,morning
4,13.884972,100.506767,2019-07-08 05:52:06,1316,Monday,July,morning
...,...,...,...,...,...,...,...
4256984,13.966694,100.712861,2019-09-01 14:00:05,1573,Sunday,September,afternoon
4256985,14.299056,100.425677,2019-09-01 16:34:45,117,Sunday,September,afternoon
4256986,13.827038,100.622365,2019-09-01 17:02:56,1373,Sunday,September,evening
4256987,13.827038,100.622365,2019-09-01 17:02:57,1373,Sunday,September,evening


In [7]:
df['month'].value_counts()

August       4142952
September     112454
July            1583
Name: month, dtype: int64

In [8]:
df_July = df[df['month'] == "July"]
df_July 

,latitude,longitude,datetime,id,day,month,period
0,13.868754,100.409435,2019-07-07 22:57:46,1319,Sunday,July,night
1,13.884972,100.506767,2019-07-08 05:52:01,1316,Monday,July,morning
2,13.884972,100.506767,2019-07-08 05:52:04,1316,Monday,July,morning
3,13.884972,100.506767,2019-07-08 05:52:05,1316,Monday,July,morning
4,13.884972,100.506767,2019-07-08 05:52:06,1316,Monday,July,morning
...,...,...,...,...,...,...,...
1578,14.032964,100.796383,2019-07-31 23:16:48,1525,Wednesday,July,night
1579,14.032921,100.796401,2019-07-31 23:21:12,1525,Wednesday,July,night
1580,13.989005,100.617905,2019-07-31 23:27:57,1226,Wednesday,July,night
1581,13.989005,100.617905,2019-07-31 23:27:58,1226,Wednesday,July,night


In [9]:
df_August = df[df['month'] == "August"]
df_August

,latitude,longitude,datetime,id,day,month,period
1583,14.032884,100.796485,2019-08-01 00:06:34,1525,Thursday,August,night
1584,14.032903,100.796623,2019-08-01 00:06:40,1525,Thursday,August,night
1585,14.032959,100.796692,2019-08-01 00:06:45,1525,Thursday,August,night
1586,14.033000,100.796719,2019-08-01 00:07:58,1525,Thursday,August,night
1587,14.033040,100.796747,2019-08-01 00:10:08,1525,Thursday,August,night
...,...,...,...,...,...,...,...
4144530,14.312443,100.395164,2019-08-31 23:59:58,1645,Saturday,August,night
4144531,13.963388,100.513404,2019-08-31 23:59:59,1197,Saturday,August,night
4144532,14.312524,100.395344,2019-08-31 23:59:59,1645,Saturday,August,night
4144533,13.891654,100.564141,2019-08-31 23:59:59,1022,Saturday,August,night


In [10]:
df_September = df[df['month'] == "September"]
df_September

,latitude,longitude,datetime,id,day,month,period
4144535,13.992660,100.652958,2019-09-01 00:00:00,1373,Sunday,September,night
4144536,13.817125,100.575467,2019-09-01 00:00:00,1204,Sunday,September,night
4144537,14.312605,100.395526,2019-09-01 00:00:00,1645,Sunday,September,night
4144538,14.312686,100.395709,2019-09-01 00:00:01,1645,Sunday,September,night
4144539,13.801327,100.633807,2019-09-01 00:00:01,417,Sunday,September,night
...,...,...,...,...,...,...,...
4256984,13.966694,100.712861,2019-09-01 14:00:05,1573,Sunday,September,afternoon
4256985,14.299056,100.425677,2019-09-01 16:34:45,117,Sunday,September,afternoon
4256986,13.827038,100.622365,2019-09-01 17:02:56,1373,Sunday,September,evening
4256987,13.827038,100.622365,2019-09-01 17:02:57,1373,Sunday,September,evening


In [11]:
df_August['day'].value_counts()

Friday       725725
Saturday     669686
Thursday     619268
Wednesday    554105
Monday       535386
Tuesday      533038
Sunday       505744
Name: day, dtype: int64

In [12]:
select_df = df.head(5000)

In [13]:
from keplergl import KeplerGl 
map_1 = KeplerGl(height=500)
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(height=500)

In [16]:
map_1.add_data(data=df_August, name='data_1')

In [14]:
map_2 = KeplerGl(height=500)
map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(height=500)

In [17]:
map_2.add_data(data=df_September, name='data_1')